In [1]:
# importing required liberaries

import os
import string
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hello\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hello\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_excel('Input.xlsx')
data.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [3]:
# Fetching data from URLs using BeautifulSoup
# Storeing data in local directory named as "Extracted_articles"

url_id_not_found = []

for url, url_id in zip(data['URL'], data['URL_ID']):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.text, 'lxml')
        
        title = soup.find('title')
        
        post_content = soup.find('div', class_='td-post-content')
        
        content = ''
        for i in post_content.find_all('p'):
            content += ' ' + i.text
        
    except requests.exceptions.RequestException as e:
        print(f'URL with url_id - {url_id} not Found.')
        url_id_not_found.append(url_id)
        
    except AttributeError as e:
        print(f'Attribute Error at URL with url_id - {url_id}')
        url_id_not_found.append(url_id)
        
    else:
        path = 'Extracted_articles'
        if not os.path.exists(path):
                !mkdir {path}
        file_path = os.path.join(path, f'{url_id}.txt')
        if not os.path.exists(file_path):
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(f'{title.text}\n{content}')


URL with url_id - blackassign0036 not Found.
URL with url_id - blackassign0049 not Found.


In [4]:
# loading all stopwords from StopWords Directory and storing it in common list

stop_word = []
for file in os.listdir('StopWords'):
    with open(os.path.join('StopWords', file), 'r') as f:
        for line in f.readlines():
            stop_word.append(line.strip().split('|')[0].strip().lower())
stop_word = set(stop_word)

In [5]:
# Loading all positive and Negative words from MasterDictionary Directory and storing it in different list

pos = []
neg = []
for file in os.listdir('MasterDictionary'):
    with open(os.path.join('MasterDictionary', file), 'r') as f:
        if file == 'positive-words.txt':
            pos.extend(f.read().splitlines())
        else:
            neg.extend(f.read().splitlines())

In [6]:
# Loading scrapped data from local directory 
# Calculating few required Features

articles = []
tokenized_articles = []
positive_score = []
negative_score = []
subjectivity_score = []
polarity_score = []

for file in os.listdir('Extracted_articles'):
    with open(os.path.join('Extracted_articles', file), 'r', encoding='utf-8') as f:
        text = f.read()
        articles.append(text)

for text in articles:
    words = nltk.word_tokenize(text)
    tokenized_articles = [word for word in words if word.lower() not in stop_word]
    p_count = sum([1 for word in tokenized_articles if word.lower() in pos])
    n_count = sum([-1 for word in tokenized_articles if word.lower() in neg])*-1
    subjectivity_score.append((p_count+n_count)/((len(tokenized_articles)) + 0.000001))
    polarity_score.append((p_count-n_count)/((p_count+n_count) + 0.000001))
    positive_score.append(p_count)
    negative_score.append(n_count)

In [7]:
# Calculatoin rest of the Features

stop_words = stopwords.words('english')
exclude = string.punctuation


def find(text):
    # removing pucntuation
    new_text = text.translate(str.maketrans('','',exclude))
    
    # word tokenizing
    new_text = nltk.word_tokenize(new_text)
    
    # removing stop_words
    words = [word for word in new_text if word.lower() not in stop_words]
    total_words = len(words)                              # WORD COUNT
    
    # sentence tokenization
    sentences = nltk.sent_tokenize(text)
    total_sentences = len(sentences)
    avg_len_of_sen = total_words/total_sentences          # AVG SENTENCE LENGTH 
    
    # Finding complex words count, syllable count, syllable word count
    complex_words = 0                                     # COMPLEX WORD COUNT
    syllable_cnt = 0
    syllable_wrd_cnt = 0
    for word in words:
        if word.endswith('es'):
            word = word[:-2]
        if word.endswith('ed'):
            word = word[:-2]
        cnt = 0
        flag = True
        for i in word.lower():
            if i in 'aeiou':
                if flag:
                    flag = False
                    syllable_wrd_cnt+=1
                syllable_cnt+=1
                cnt+=1
        if cnt>2:
            complex_words+=1
    
    complex_words_in_pct = complex_words/total_words      # PERCENTAGE OF COMPLEX WORDS
    fi = 0.4 * (avg_len_of_sen+complex_words_in_pct)      # FOG INDEX
    avg_no_of_wrds_per_sen = total_words/total_sentences  # AVG NUMBER OF WORDS PER SENTENCE
    syll_per_wrd = syllable_cnt/syllable_wrd_cnt          # SYLLABLE PER WORD
    
    total_char = sum([len(word) for word in words])
    avg_wrd_len = total_char/total_words                  # AVG WORD LENGTH
    
    return [avg_len_of_sen, complex_words_in_pct, fi, avg_no_of_wrds_per_sen, complex_words, total_words, syll_per_wrd, avg_wrd_len]
    
def count_personal_pronouns(text):
    # regex pattern to match personal pronouns
    pattern = r'\b(?:I|we|my|ours|us)\b'

    matches = re.findall(pattern, text, flags=re.IGNORECASE)

    # exclude "US" as a country name
    country_pattern = r'\b(?:US)\b'
    country_matches = re.findall(country_pattern, text, flags=re.IGNORECASE)
    for country_match in country_matches:
        matches = [match for match in matches if match != 'US']

    # Count the number of matches
    count = len(matches)
    return count


In [8]:
avg_sentence_length = []
Percentage_of_complex_words = []
Fog_index = []
Avg_no_of_words_per_sentence = []
word_count = []
complex_word_count = []
personal_pronoun_count = []
syllable_per_word = []
avg_word_len = []

# Function calling for each arcicle to calculate required features

for text in articles:
    lst = find(text)
    avg_sentence_length.append(lst[0])
    Percentage_of_complex_words.append(lst[1])
    Fog_index.append(lst[2])
    Avg_no_of_words_per_sentence.append(lst[3])
    word_count.append(lst[5])
    complex_word_count.append(lst[4])
    syllable_per_word.append(lst[6])
    avg_word_len.append(lst[7])
    personal_pronoun_count.append(count_personal_pronouns(text))

In [9]:
# read "Output Data Structure.xlsx" File

output = pd.read_excel('Output Data Structure.xlsx')
output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# drop rows for which url not found

output = output[~output['URL_ID'].isin(url_id_not_found)]

In [12]:
# replacing data with calculated data

output['POSITIVE SCORE'] = positive_score
output['NEGATIVE SCORE'] = negative_score
output['POLARITY SCORE'] = polarity_score
output['SUBJECTIVITY SCORE'] = subjectivity_score
output['AVG SENTENCE LENGTH'] = avg_sentence_length
output['PERCENTAGE OF COMPLEX WORDS'] = Percentage_of_complex_words
output['FOG INDEX'] = Fog_index
output['AVG NUMBER OF WORDS PER SENTENCE'] = Avg_no_of_words_per_sentence
output['COMPLEX WORD COUNT'] = complex_word_count
output['WORD COUNT'] = word_count
output['SYLLABLE PER WORD'] = syllable_per_word
output['PERSONAL PRONOUNS'] = personal_pronoun_count
output['AVG WORD LENGTH'] = avg_word_len


In [13]:
output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,2,1,0.333333,0.015707,7.560000,0.365079,3.170032,7.560000,69,189,2.358696,4,6.253968
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,50,31,0.234568,0.093318,11.220779,0.474537,4.678127,11.220779,410,864,2.708785,4,7.120370
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36,23,0.220339,0.077530,12.071429,0.572485,5.057566,12.071429,387,676,2.971386,13,8.038462
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,37,74,-0.333333,0.148198,13.274510,0.548006,5.529006,13.274510,371,677,2.858877,5,7.741507
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18,8,0.384615,0.062802,10.717949,0.447368,4.466127,10.717949,187,418,2.639024,6,7.208134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,26,54,-0.350000,0.126582,13.140000,0.435312,5.430125,13.140000,286,657,2.581903,4,6.989346
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,35,-0.228070,0.110251,14.105263,0.397388,5.801060,14.105263,213,536,2.378846,7,6.348881
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,1,0,0.999999,0.009524,20.200000,0.455446,8.262178,20.200000,46,101,2.280000,0,6.663366
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,11,3,0.571429,0.046980,11.357143,0.317610,4.669901,11.357143,101,318,2.260726,4,6.028302


In [14]:
# save output as csv file

output.to_csv('Output_Data.csv')